This notebook by [Kel Markert](https://github.com/KMarkert) walks through the use of Earth Engine within a Python environment to highlight combining different data sources from Earth Engine for visualization and analytics in the space of climate impact on commodity crops.

In [ ]:
# Authenticate and Initialize Earth Engine
import ee
# Authenticate as your Qwiklabs User
ee.Authenticate()
# Initialize the library by specifying the GCP project
your_project_id = ''
ee.Initialize(project=your_project_id)

In [ ]:
# import packages
import google
from google.colab import auth

import ee
import geemap
import geemap.colormaps as cm

In [ ]:
# create a Map object with geemap to visualize EE results
m = geemap.Map()

In [ ]:
# programically create a scratch cell for displaying the Map
from google.colab import _frontend
_frontend.create_scratch_cell("#@title Map\nm", False)

## Flooding data

Southeast Asia experiences regular flooding every year and is a large rice production region. Flooding affects rice production and can be useful for determining shifts in rice markets.

In [ ]:
admin_lvl1 = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level1')

aoi = admin_lvl1.filter(ee.Filter.eq('ADM0_NAME', 'Cambodia'))

m.center_object(aoi, 7);  # country of interest

In [ ]:
# define the time period to view
start_date = '2023-10-08'
end_date = '2023-10-18'

In [ ]:
# load in the Sentinel 1 dataset for time and location
s1 = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterDate(start_date, end_date)
    .filterBounds(aoi)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
)

# create a composite from the data
s1_composite = s1.mean()

In [ ]:
# calculate water area from the satellite imagery
# uses a simple thresholding technique
water = s1_composite.select('VH').lte(-18).selfMask().rename('water')

In [ ]:
# add S1 composite layer
m.add_layer(s1_composite, {'bands': 'VH', 'min': -30, 'max': -5}, 'S1 Composite')

# add surface water layer
m.add_layer(water, {'min': 0,'max': 1, 'palette':['white', 'darkblue']}, 'Surface Water')

## Weather Forecasts

Flooding is dependant on rainfall and by understanding what future rainfall patterns will be.

In [ ]:
# load in the Global Forecast System data
# select to the precipitation forecast
gfs_precip = (
    ee.ImageCollection("NOAA/GFS0P25")
    .select('total_precipitation_surface')
)

In [ ]:
# get the end date of flood layer and set  when forecasts start
forecast_date = ee.Date(end_date)

# filter the forecast data to the 5 day forecast from
precip_forecast = (
    gfs_precip
    # filter by model initalization
    .filterDate(forecast_date,forecast_date.advance(6,'hour'))
    # filter to 5 day forecast
    .filter(ee.Filter.And(
        ee.Filter.gt('forecast_hours',0),
        ee.Filter.lte('forecast_hours',72)
    ))
)

In [ ]:
# inspect the result to verify our filtering is correct
precip_forecast

In [ ]:
# calculate the accumulated precip forecast for the next five days
precip_total = precip_forecast.sum().resample('bicubic').clip(aoi)

In [ ]:
# add the accumulated precip image to the map
m.add_layer(precip_total, {'min': 25,'max': 150, 'palette':cm.get_palette('viridis')}, 'Forecasted precip [mm]', True, 0.65)

## Crop information

Crop area is indicative of production and understanding impacts of floods can help assess production losses as well as economic impacts.

In [ ]:
# load in the dynamic world land cover dataset
# filter to location and year of interest
dynamic_world = (
    ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
    .filterBounds(aoi)
    .filterDate("2023-01-01", "2024-01-01")
)

In [ ]:
# create a land cover composite and select crop areas
crops = dynamic_world.select('label').mode().eq(4)

# indentify intersection of crops and surface water
inundated_crops = crops.And(water)

In [ ]:
# add crop layer to map
m.add_layer(crops.selfMask(), {'min': 0,'max': 1, 'palette':['white', 'orange']}, 'Crops', False)

# add inundated crop layer to map
m.add_layer(inundated_crops.selfMask(), {'min': 0,'max': 1, 'palette':['white', 'Red']}, 'Inundated Crops')

## Analytics

Up until now Earth Engine has been used for calculating geospatial data and visualizing it. Now, that geospatial data will be distilled to information that can be used for a decision.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# load in packages for charting and local data structure
import altair as alt
import pandas as pd

In [ ]:
# add the Admin features to the map for visualization
m.add_layer(
    aoi.style(
        color= '000000',
        width= 2,
        fillColor= 'ffffff00',  # with alpha set for partial transparency
    ),
    {},
    'Admin Lvl 1'
)


In [ ]:
#@title Time series function
def get_timeseries(province):
    """Calcualte a time series of average temperature for a given US state

       args:
           state_name (str): name of the state to get temperature time series

        returns:
            pd.DataFrame: table of time series for yearly average temperature
    """

    def calc_ts(t):
        # determine start and end time for the reduction
        t1 = ee.Date(t)
        t2 = t1.advance(1, 'hour')

        # get the first image within time range
        img = precip_forecast.filter(ee.Filter.eq('forecast_time', t)).first()

        # apply the reduction for the areal avg precip
        stat = img.reduceRegion(
            geometry=province.geometry(),
            reducer=ee.Reducer.mean(),
            scale=img.projection().nominalScale(),
            tileScale=2
        )

        # update the reduction result with time info
        stat = stat.combine({
            'date': t1.format('YYYY-MM-dd HH:mm:ss'),
            'system:time_start': t1.millis()
        })
        return province.set(stat).setGeometry(None)

    # get the years for a time series
    datetimes = precip_forecast.aggregate_array('forecast_time')

    # map over all of the forecast times and convert to FeatureCollection
    ts = ee.FeatureCollection(datetimes.map(calc_ts))

    # request that the table be returned as pd.DataFrame
    return ee.data.computeFeatures({
        'expression': ts,
        'fileFormat': 'PANDAS_DATAFRAME'
    })

In [ ]:
#@title Select province for summary stats
provinces = aoi.aggregate_array("ADM1_NAME").getInfo()

Dropdown_ = widgets.Dropdown(
    options=provinces,
    description='Select Province',
)
output = widgets.Output()

display(Dropdown_)

In [ ]:
# get the selected province
province_name = Dropdown_.value
province_feature = ee.Feature(
    aoi.filter(ee.Filter.eq('ADM1_NAME', province_name)).first()
)

In [ ]:
# request the precip forecast time series for province
ts_data = get_timeseries(province_feature)

In [ ]:
# plot the data as a timeseries
alt.Chart(ts_data).mark_area().encode(
    x=alt.X('date:T', axis=alt.Axis(title='Datetime', format="%Y-%m-%d %H:%M:%S")),
    y=alt.Y('total_precipitation_surface:Q',axis=alt.Axis(title='Precipitation [mm]')),
    tooltip=[
            alt.Tooltip('date:T', title='Datetime', format="%Y-%m-%d %H:%M:%S"),
            alt.Tooltip('total_precipitation_surface:Q', title='Precipitation [mm]')
    ]
).properties(
    width=700,
    height=300
)

In [ ]:
# combine surface water and precip forecast images
hydro_layers = water.unmask(0).addBands(precip_total.rename('precip'))

# calculate the average for both surface water and precip
# for the province area
hydro_stats = hydro_layers.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=province_feature.geometry(),
    scale=100,
)

In [ ]:
# get the area of crops for the selected province
inundated_crop_area = (
    inundated_crops
    # convert binary image to area
    .multiply(ee.Image.pixelArea())
    # convert to sq km
    .multiply(1e-6)
    # get the sum of crop area inundated
    .reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=province_feature.geometry(),
        scale=100,
    )
).get('label')

In [ ]:
# print the summary statistics
print(f'Area avg forecasted precip: {hydro_stats.get("precip").getInfo():.2f} mm')
print(f'% Area covered by water: {hydro_stats.get("water").getInfo()*100:.2f} %')
print(f'Inundated crop area: {inundated_crop_area.getInfo():.2f} sq km')